In [1]:
import pysynth as ps
from pprint import pprint

<h1>Converting midi to numpy array in python</h1>
https://medium.com/analytics-vidhya/convert-midi-file-to-numpy-array-in-python-7d00531890c

In [1]:
import mido
import string
import numpy as np
mid = mido.MidiFile('./midi/sonata1.mid', clip=True)
mid.tracks

[MidiTrack([
   MetaMessage('track_name', name='Piano Template', time=0),
   MetaMessage('text', text='By <Name>', time=0),
   MetaMessage('copyright', text='Copyright Â© <Year> by <Name>', time=0),
   MetaMessage('copyright', text='A', time=0),
   MetaMessage('set_tempo', tempo=428571, time=0),
   MetaMessage('key_signature', key='D', time=0),
   MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
   MetaMessage('end_of_track', time=0)]),
 MidiTrack([
   MetaMessage('midi_port', port=0, time=0),
   MetaMessage('track_name', name='Right Hand', time=0),
   Message('program_change', channel=0, program=0, time=0),
   Message('pitchwheel', channel=0, pitch=0, time=0),
   Message('control_change', channel=0, control=7, value=127, time=0),
   Message('control_change', channel=0, control=10, value=64, time=0),
   Message('note_on', channel=0, note=74, velocity=110, time=0),
   Message('note_on', channel=0, note=78, velocity=11

In [11]:
def msg2dict(msg):
#     print(msg)
    result = dict()
    if 'note_on' in msg:
        on_ = True
    elif 'note_off' in msg:
        on_ = False
    else:
        on_ = None
    result['time'] = int(msg[msg.rfind('time'):].split(' ')[0].split('=')[1].translate(
        str.maketrans({a: None for a in string.punctuation})))

    if on_ is not None:
        for k in ['note', 'velocity']:
            result[k] = int(msg[msg.rfind(k):].split(' ')[0].split('=')[1].translate(
                str.maketrans({a: None for a in string.punctuation})))
    return [result, on_]

In [12]:
def get_new_state(new_msg):
    new_msg, on_ = msg2dict(str(new_msg))
#     print(new_msg)
    return new_msg
def track2seq(track):
    # piano has 88 notes, corresponding to note id 21 to 108, any note out of the id range will be ignored
    result = []
    for i in range(1, len(track)):
        new_msg = get_new_state(track[i])
        result.append(new_msg)
    return result

In [13]:
def mid2arry(mid, min_msg_pct=0.1):
    tracks_len = [len(tr) for tr in mid.tracks]
    min_n_msg = max(tracks_len) * min_msg_pct
    # convert each track to nested list
    all_arys = []
    for i in range(len(mid.tracks)):
        if len(mid.tracks[i]) > min_n_msg:
            ary_i = track2seq(mid.tracks[i])
#             print(ary_i)
            all_arys.append(ary_i)
    return all_arys

In [14]:
result_array = mid2arry(mid)
# result_array[0][5]['time']
song = []
note_list = []
buffer = []
msg_idx = 0

#track1
for i in range(len(result_array[0])):
    if 'note' in result_array[0][i]:
        if result_array[0][i]['time'] == 0:
            buffer.append(i)
        else:
            if buffer:
                for idx in buffer:
                    note_list.append(result_array[0][idx]['note'])
                buffer.clear()
            note_list.append(result_array[0][i]['note'])
#             note_list.append(result_array[0][i]['time'])
            note_list.insert(0, result_array[0][i]['time'])
            
            song.append(list(note_list))
            note_list.clear()
            
# for i in range(len(song)):
#     print(song[i])
        

In [15]:
result_array = mid2arry(mid)
# result_array[0][5]['time']
song = []

# track1
for i in range(len(result_array[0])):
    if 'note' in result_array[0][i] and result_array[0][i]['time'] > 0:
        song.append([result_array[0][i]['note'], result_array[0][i]['time']])
    

# for i in range(len(song)):
#     print(song[i])

# Run MarkovChain

In [15]:
from src.MarkovMusic_hyunseo import MusicMatrix

In [16]:
matrix = MusicMatrix(song)

start_note = [60, 240]

random_song = []
for i in range(0, 100):
    start_note = matrix.next_note(start_note)
    random_song.append(start_note)
    print(start_note)


38 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 

38 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 2 0 0 0 0 0 0 0 2 0 0 0 

40 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 

41 0 4 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 

42 0 0 0 0 0 0 2 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

43 0 0 6 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

44 0 0 0 0 0 0 2 0 2 0 0 2 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 

45 0 0 0 2 4 0 0 2 2 2 4 6 0 0 2 0 2 0 0 0 0 0 0 0 0 0 0 

46 2 0 0 0 2 0 4 0 0 2 0 6 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 

47 0 0 0 0 0 2 4 0 0 0 2 2 0 2 0 0 0 0 0 0 0 0 0 2 0 0 0 

48 0 0 0 0 0 0 4 8 4 0 0 2 0 2 0 0 0 0 0 2 0 0 0 0 0 0 0 

49 2 0 0 0 0 0 0 0 2 0 0 8 0 4 0 0 0 0 2 0 0 0 0 0 0 0 0 

50 2 0 2 0 0 6 0 4 2 10 6 8 6 10 8 2 6 4 0 0 0 2 0 0 0 0 0 

51 0 0 0 0 0 0 0 0 0 6 0 2 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 

52 0 0 0 0 0 0 2 2 0 2 0 18 0 2 16 0 6 4 4 0 0 0 4 0 0 2 0 

53 0 0 0 0 2 0 0 0 0 0 2 14 0 22 0 0 6 0 12 0 0 0 0 0 0 0 0 

54 0 0 0 0 0 0 0 0 0 0 0 

# Undertail

In [17]:
song = [('g#3', 16.0), ('e4', 16.0), ('d#4', 16.0), ('d4', 16.0), ('d#4', 16.0), ('r', 16.0), ('c#4', 16.0), ('b3', 16.0), ('a#3', 16.0), ('r', 16.0), ('g#3', 16.0), ('g3', 16.0), ('g#3', 16.0), ('r', 16.0), ('d#3', 16.0), ('e3', 16.0), ('d#3', 16.0), ('e3', 16.0), ('d#3', 16.0), ('d3', 16.0), ('d#3', 16.0), ('g3', 16.0), ('b3', 16.0), ('a#3', 16.0), ('g#3', 16.0), ('r', 16.0), ('g#3', 16.0), ('a#3', 16.0), ('b3', 16.0), ('r', 16.0), ('a#3', 16.0), ('b3', 16.0), ('c#4', 16.0), ('r', 16.0), ('b3', 16.0), ('a#3', 16.0), ('g3', 16.0), ('r', 16.0), ('g3', 16.0), ('a#3', 16.0), ('b3', 16.0), ('r', 16.0), ('a#3', 16.0), ('g#3', 16.0), ('d#3', 16.0), ('r', 16.0), ('d#3', 16.0), ('e3', 16.0), ('d#3', 16.0), ('e3', 16.0), ('d#3', 16.0), ('d3', 16.0), ('d#3', 16.0), ('g3', 16.0), ('b3', 16.0), ('a#3', 16.0), ('g#3', 16.0), ('r', 16.0), ('g#3', 16.0), ('g3', 16.0), ('g#3', 16.0), ('r', 16.0), ('g#4', 16.0)]

# ps.make_wav(song, fn='examples/undertail.wav')

matrix = MusicMatrix(song)

start_note = ['d4', 16]

random_song = []
for i in range(0, 100):
    start_note = matrix.next_note(start_note)
    random_song.append(start_note)
    print(start_note)

# ps.make_wav(random_song, fn='examples/random_undertail.wav')

# make_midi(midi_path='midi/random_undertail.mid', notes=random_song)

['d#4', 16.0]
['r', 16.0]
['g3', 16.0]
['r', 16.0]
['a#3', 16.0]
['g#3', 16.0]
['r', 16.0]
['a#3', 16.0]
['b3', 16.0]
['a#3', 16.0]
['g#3', 16.0]
['r', 16.0]
['a#3', 16.0]
['g3', 16.0]
['b3', 16.0]
['a#3', 16.0]
['g#3', 16.0]
['r', 16.0]
['a#3', 16.0]
['r', 16.0]
['g#3', 16.0]
['g3', 16.0]
['r', 16.0]
['d#3', 16.0]
['e3', 16.0]
['d#3', 16.0]
['d3', 16.0]
['d#3', 16.0]
['r', 16.0]
['g#3', 16.0]
['r', 16.0]
['d#3', 16.0]
['e3', 16.0]
['d#3', 16.0]
['g3', 16.0]
['b3', 16.0]
['r', 16.0]
['b3', 16.0]
['a#3', 16.0]
['g#3', 16.0]
['r', 16.0]
['d#3', 16.0]
['e3', 16.0]
['d#3', 16.0]
['g3', 16.0]
['g#3', 16.0]
['r', 16.0]
['a#3', 16.0]
['b3', 16.0]
['a#3', 16.0]
['b3', 16.0]
['a#3', 16.0]
['b3', 16.0]
['c#4', 16.0]
['b3', 16.0]
['r', 16.0]
['g#3', 16.0]
['g3', 16.0]
['b3', 16.0]
['a#3', 16.0]
['g#3', 16.0]
['r', 16.0]
['g3', 16.0]
['a#3', 16.0]
['g3', 16.0]
['g#3', 16.0]
['r', 16.0]
['g3', 16.0]
['r', 16.0]
['a#3', 16.0]
['g#3', 16.0]
['a#3', 16.0]
['b3', 16.0]
['r', 16.0]
['b3', 16.0]
['r', 16

# Debussy - Reverie

In [12]:
song = [('g5', 2.0), ('d5', 1.3333333333333333), ('e5', 8.0), ('f5', 8.0), ('g5', 4.0), ('e5', 8.0), ('d5', 8.0), ('e5', 5.333333333333333), ('c5', 5.314878892780648), ('e5', 5.314878892710022), ('d5', 1.0), ('a#4', 4.0), ('d5', 4.0), ('e5', 4.0), ('f5', 4.0), ('c5', 1.0), ('g4', 2.0), ('a4', 0.4999999999999998), ('a5', 2.0), ('e5', 1.3333333333333333), ('c5', 8.0), ('e5', 8.0), ('d5', 4.0), ('a#4', 8.0), ('g4', 8.0), ('a5', 2.0), ('e5', 1.3333333333333333), ('c5', 8.0), ('e5', 8.0), ('d5', 4.0), ('a#4', 8.0), ('g4', 8.0), ('g5', 2.0), ('d5', 1.3333333333333333), ('a#4', 8.0), ('d5', 8.0), ('c5', 4.0), ('a4', 8.0), ('g4', 8.0), ('a4', 4.0), ('e4', 8.0), ('a4', 8.0), ('f4', 4.0), ('d4', 8.0), ('f4', 8.0), ('d4', 2.0), ('c4', 2.000000000000007), ('c6', 2.0), ('g5', 1.3333333333333333), ('a5', 8.0), ('a#5', 8.0), ('c6', 4.0), ('a5', 8.0), ('g5', 8.0), ('a5', 5.333333333333333), ('f5', 5.314878892498194), ('a5', 5.314878892498194), ('g5', 1.0), ('d#5', 4.0), ('g5', 4.0), ('a5', 4.0), ('a#5', 4.0), ('f5', 1.0), ('a5', 4.0), ('a#5', 4.0), ('c6', 4.0), ('d6', 4.0), ('a5', 1.0), ('a#5', 4.0), ('d6', 4.0), ('f6', 2.0), ('c#6', 2.0000000003000054), ('r', 8.084210527894808), ('d5', 8.0), ('e5', 8.0), ('r', 4.0), ('f5', 8.0), ('a5', 8.0), ('f6', 2.0), ('c#6', 2.0), ('r', 4.0), ('d5', 8.0), ('e5', 8.0), ('r', 4.0), ('f5', 8.0), ('a5', 8.0), ('f5', 4.0), ('d4', 8.0), ('e4', 8.0), ('f5', 0.3333333333333333), ('f4', 8.0), ('a4', 8.0), ('f4', 2.0), ('f4', 1.0), ('d4', 4.0), ('c4', 4.0), ('d4', 1.0), ('c5', 4.0), ('a#4', 8.0), ('a4', 8.0), ('g4', 1.3333333333333333), ('a4', 8.0), ('a#4', 8.0), ('c5', 4.0), ('c5', 8.0), ('d#5', 8.0), ('d5', 0.5), ('c5', 4.0), ('a#4', 8.0), ('a4', 8.0), ('g4', 1.3333333333333333), ('a4', 8.0), ('a#4', 8.0), ('c5', 8.0), ('d#5', 8.0), ('e5', 0.6666666666666666), ('d#5', 4.0), ('e5', 4.0), ('g5', 4.0), ('f5', 8.0), ('e5', 8.0), ('d5', 2.0), ('d5', 2.0), ('e5', 4.0), ('c5', 0.08888888888888886), ('f5', 4.0), ('e5', 8.0), ('d5', 8.0), ('c5', 1.0), ('d5', 4.0), ('f5', 0.5), ('e5', 4.0), ('d5', 8.0), ('c5', 8.0), ('a#4', 2.0), ('a#5', 2.0), ('a5', 4.0), ('g5', 4.0), ('e5', 0.4444444444444444), ('f5', 0.6666666666666666), ('f4', 4.0), ('f5', 0.23529411764705882), ('e4', 4.0), ('f4', 16.0), ('a4', 16.0), ('f4', 16.0), ('e4', 4.0), ('d4', 4.0), ('c4', 2.0), ('e4', 4.0), ('c#4', 4.0), ('c#4', 4.0), ('e4', 4.0), ('f#4', 4.0), ('g#4', 4.0), ('e4', 2.0), ('f#5', 4.0), ('g#5', 4.0), ('e5', 4.0), ('c#6', 4.0), ('f#5', 4.0), ('g#5', 16.0), ('f#5', 16.0), ('g#5', 16.0), ('e5', 4.0), ('b4', 4.0), ('g#4', 4.0), ('b4', 4.0), ('g#4', 4.0), ('e4', 4.0), ('f#4', 2.0), ('c#5', 2.0), ('f#5', 4.0), ('g#5', 4.0), ('e5', 4.0), ('c#6', 4.0), ('f#5', 4.0), ('g#5', 16.0), ('f#5', 16.0), ('g#5', 16.0), ('e5', 4.0), ('b5', 4.0), ('c#6', 4.0), ('e6', 4.0), ('d#6', 4.0), ('b5', 4.0), ('c#6', 4.0), ('e6', 4.0), ('f#6', 4.0), ('b5', 4.000000000000114), ('c#6', 4.0), ('e6', 4.0), ('d#6', 4.0), ('b5', 4.0), ('c#6', 4.0), ('e6', 4.0), ('f#6', 4.0), ('b5', 4.0), ('g6', 1.3333333333333333), ('a5', 4.0), ('g5', 2.6666666666666665), ('a4', 8.0), ('g4', 2.6666666666666665), ('a4', 8.0), ('d4', 4.0), ('e4', 4.0), ('c4', 4.0), ('a4', 4.0), ('d4', 4.0), ('e4', 16.0), ('d4', 16.0), ('e4', 16.0), ('c4', 2.0), ('d5', 4.0), ('e5', 16.0), ('d5', 16.0), ('e5', 16.0), ('c5', 2.0), ('g5', 2.0), ('d5', 1.3333333333333333), ('e5', 8.0), ('f5', 8.0), ('g5', 4.0), ('e5', 8.0), ('d5', 8.0), ('e5', 5.333333333333333), ('c5', 5.31487889037946), ('e5', 5.31487889037946), ('d5', 1.0), ('a#4', 4.0), ('d5', 4.0), ('e5', 4.0), ('f5', 4.0), ('c5', 1.0), ('g4', 2.0), ('a4', 0.5), ('a5', 2.0), ('e5', 1.3333333333333333), ('c5', 8.0), ('e5', 8.0), ('d5', 4.0), ('a#4', 8.0), ('g4', 8.0), ('a5', 2.0), ('e5', 1.3333333333333333), ('c5', 8.0), ('e5', 8.0), ('d5', 4.0), ('a#4', 8.0), ('g4', 8.0), ('g5', 2.0), ('d5', 1.3333333333333333), ('a#4', 8.0), ('d5', 8.0), ('c5', 4.0), ('a4', 8.0), ('f4', 8.0), ('g5', 2.0), ('d5', 1.3333333333333333), ('a#4', 8.0), ('d5', 8.0), ('c5', 4.0), ('d5', 4.0), ('a#4', 4.0), ('e4', 0.02580385041830461), ('a4', 4.0), ('a4', 2.0), ('a#4', 4.0), ('a4', 4.0), ('a4', 2.0), ('a#4', 4.0), ('a#4', 0.6666666666666666), ('a4', 4.0), ('a#4', 16.0), ('d5', 16.0), ('a#4', 16.0), ('a4', 4.0), ('g4', 4.0), ('f4', 2.0), ('a#5', 4.0), ('a4', 0.49983729252717796), ('a5', 4.0), ('a5', 2.0), ('a#5', 4.0), ('a#5', 2.0), ('a5', 2.0), ('a#5', 4.0), ('a5', 4.0), ('a#5', 16.0), ('d6', 16.0), ('a#5', 16.0), ('a5', 2.0), ('g5', 2.0), ('a5', 0.36355029584697357), ('a6', 1.0)]

# ps.make_wav(song[:100], fn='examples/Debussy_Reverie.wav')

matrix = MusicMatrix(song)

start_note = ['g5', 2]

random_song = []
for i in range(0, 100):
    start_note = matrix.next_note(start_note)
    random_song.append(start_note)

# ps.make_wav(random_song, fn='examples/random_debussy.wav')
# make_midi(midi_path='midi/random_debussy.mid', notes=random_song)

KeyError: '0.04'

# Synth Solo

In [12]:
song = [('d3', 16.0), ('r', 16.0), ('g3', 16.0), ('r', 16.0), ('a3', 16.0), ('r', 16.0), ('d3', 16.0), ('r', 16.0), ('g3', 16.0), ('r', 16.0), ('a3', 16.0), ('r', 16.0), ('d3', 16.0), ('r', 16.0), ('d4', 16.0), ('r', 16.0), ('b3', 16.0), ('r', 16.0), ('d3', 16.0), ('r', 16.0), ('a3', 16.0), ('r', 16.0), ('g3', 16.0), ('r', 16.0), ('d3', 16.0), ('r', 16.0), ('a3', 16.0), ('r', 16.0), ('g3', 16.0), ('r', 16.0), ('d3', 16.0), ('r', 16.0), ('r', 5.333333333333333), ('f#3', 16.0), ('g3', 16.0), ('r', 16.0), ('d3', 16.0), ('r', 16.0), ('g3', 16.0), ('r', 16.0), ('a3', 16.0), ('r', 16.0), ('d3', 16.0), ('r', 8.0), ('f#3', 16.0), ('r', 16.0), ('f#3', 16.0), ('g3', 16.0), ('r', 16.0), ('a3', 16.0), ('r', 16.0), ('d3', 16.0), ('r', 16.0), ('b3', 16.0), ('r', 16.0), ('a3', 16.0), ('r', 16.0), ('g3', 16.0), ('r', 16.0), ('d3', 16.0), ('r', 16.0), ('f#3', 16.0), ('r', 16.0), ('g3', 16.0), ('r', 16.0), ('e3', 16.0), ('r', 16.0), ('g3', 16.0), ('r', 16.0), ('d4', 16.0), ('r', 16.0), ('f#3', 16.0), ('r', 16.0), ('g3', 16.0), ('r', 16.0), ('a3', 16.0), ('r', 16.0), ('f#3', 16.0), ('r', 16.0), ('g3', 16.0), ('r', 16.0), ('f#3', 16.0), ('r', 16.0), ('a3', 16.0), ('r', 16.0), ('g3', 16.0), ('r', 16.0), ('f#4', 16.0), ('r', 16.0), ('g4', 16.0), ('r', 16.0), ('g3', 16.0), ('r', 16.0), ('b3', 16.0), ('r', 16.0), ('d4', 16.0)]

# ps.make_wav(song[:100], fn='examples/synth_solo.wav')

matrix = MusicMatrix(song)

start_note = ['d4', 16]

random_song = []
for i in range(0, 100):
    start_note = matrix.next_note(start_note)
    random_song.append(start_note)

# ps.make_wav(random_song, fn='examples/random_synth_solo.wav')
make_midi(midi_path='midi/random_synth_solo.mid', notes=random_song)

# Mix Songs

In [17]:
song1 = [['c4', 4], ['c4', 4], ['c4', 4], ['d4', 8], ['e4', 4], ['e4', 4], ['d4', 8], ['e4', 4], ['f4', 8], ['g4', 2], ['c4', 8], ['c4', 8], ['c4', 8], ['g4', 8], ['g4', 8], ['g4', 8], ['e4', 8], ['e4', 8], ['e4', 8], ['c4', 8], ['c4', 8], ['c4', 8], ['g4', 4], ['f4', 8], ['e4', 4], ['d4', 8], ['c4', 2]]

song2 = [('g#3', 16.0), ('e4', 16.0), ('d#4', 16.0), ('d4', 16.0), ('d#4', 16.0), ('r', 16.0), ('c#4', 16.0), ('b3', 16.0), ('a#3', 16.0), ('r', 16.0), ('g#3', 16.0), ('g3', 16.0), ('g#3', 16.0), ('r', 16.0), ('d#3', 16.0), ('e3', 16.0), ('d#3', 16.0), ('e3', 16.0), ('d#3', 16.0), ('d3', 16.0), ('d#3', 16.0), ('g3', 16.0), ('b3', 16.0), ('a#3', 16.0), ('g#3', 16.0), ('r', 16.0), ('g#3', 16.0), ('a#3', 16.0), ('b3', 16.0), ('r', 16.0), ('a#3', 16.0), ('b3', 16.0), ('c#4', 16.0), ('r', 16.0), ('b3', 16.0), ('a#3', 16.0), ('g3', 16.0), ('r', 16.0), ('g3', 16.0), ('a#3', 16.0), ('b3', 16.0), ('r', 16.0), ('a#3', 16.0), ('g#3', 16.0), ('d#3', 16.0), ('r', 16.0), ('d#3', 16.0), ('e3', 16.0), ('d#3', 16.0), ('e3', 16.0), ('d#3', 16.0), ('d3', 16.0), ('d#3', 16.0), ('g3', 16.0), ('b3', 16.0), ('a#3', 16.0), ('g#3', 16.0), ('r', 16.0), ('g#3', 16.0), ('g3', 16.0), ('g#3', 16.0), ('r', 16.0), ('g#4', 16.0)]

song = list(song1 * 2) + list(song2)

matrix = MusicMatrix(song)

pprint(matrix._markov._matrix)
pprint(matrix._timings._matrix)

start_note = ['e3', 8]

random_song = []
for i in range(0, 500):
    start_note = matrix.next_note(start_note)
    random_song.append(start_note)

# ps.make_wav(random_song, fn='examples/random_mix.wav')
make_midi(midi_path='midi/random_mix.mid', notes=random_song)

ValueError: invalid literal for int() with base 10: 'c4'